<h1><font color="blue">Tropical Restaurant in Toronto</font></h1>

This project is about evaluate the strengths and weaknesses of opening a tropical restaurant in Toronto.

### Table of contents

- [Introduction / Business Understanding](#introduction)
- [Data](#data)
- [Methodology](#methodology)
- [Analysis](#analysis)
- [Conclusion](#conclusion)


## Introduction / Business Understanding<a name="introduction"></a>

### Situation description

Noor is young African chef that is very passionate about cooking and gastronomy. After an holiday trip, 3 years ago, to visit a friend, he fell in love with Toronto and decided to settle over there and open a restaurant.

Noor is interested in gastronomy from all over the world and enforced that interest by many internships in China, Mongolia, Thailand, Cambodge, Brazil, Bolivie, Peru and in many countries in Africa like Ivory Coast, Benin, Togo, Cameroon, Senegal, Bissau, Namibia, Ethiopia and Mozambique. His gastronomy's richness is for him, an element that will open his restaurant a path to success.

After having closed each of the steps of his project, from gathering the funds to the obtention of the necessary permits, now comes the time where he has to decide where his restaurant will be settled.


### Problem presentation 


In order to find the right place for his restaurant, Noor needs to be familiar with the city and its surroundings and has hints on the affordable trendings places available. As a newcomer, he's obviously unable to have these informations.

However, despite not knowing very well the city, he knows the kinds of neighborhoods he's looking for in order to maximize his chances of making a positive impact of his customer base. Here are his criteria (in disorder) :
<font color="blue">
- a venue in a cosmopilite surroundings, where you can see people from several origins,
- a venue not in a residential neighborhood, but close to business places, to make it possible for customers to come lunch or order and be delivered in competitive durations,
- a venue in a safe neighborhood,
- a place where there can have safe night activities, especially in weekends. 
</font>

The important question here is : what would be the adequate place for Noor to settle his restaurant and make an great first impression on his customer base?


## Data<a name="data"></a>

To solve Noor's problem, we will need to gather Toronto location data and put them in competition in order to fit the criteria he expoeses in the best way possible. 

__Foursquare__ database appears to be the  adequate tool to help in the resolution of our problem.



### Exploratory analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import folium

print("Libraries imported succesfully ! ")

Libraries imported succesfully ! 


### Retrieving canadian postal codes data from a previously downloaded file.

In [2]:
df_postal = pd.read_csv("Canadian_postal_codes.csv")
df_postal.drop(["Unnamed: 0"], axis=1, inplace=True)
df_postal.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Retrieving the coordinates of each canadian postal codes

In [3]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.columns = ['Postal code', 'Latitude', 'Longitude']
df_coord.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the postal codes and their coordinates in one dataframe

In [4]:
df_toronto = pd.merge(df_postal,
                     df_coord[['Postal code','Latitude','Longitude']],
                     on='Postal code')

In [5]:
df_toronto.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [6]:
df_toronto.shape

(103, 5)

### Recovering Toronto coordinates

In [8]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Visualizing the neighborhoods on the map of Toronto

In [9]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Defining the credentials for FourSquare 

In [26]:
# FourSquare 

CLIENT_ID = '5BN44E0RZE5WZ2O3AEHZITCLUZEVWIQ1VW2SJF3YIEOZV0SV'
CLIENT_SECRET = 'T0GYPQPSHMOES5J34VIYJVYF1VICCMWWDAEVZEKTPLPQ4F2W'
version = '20200517'

radius = 500
LIMIT = 100

### Declaring a function that gather the venues around a (latitude, longitude) coordinate

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        try:
            
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                version, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        except:
            print("An error occurred on the process. Continuing with the next")
            pass

    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
An error occurred on the process. Continuing with the next
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
An error occurred on the process. Continuing with the next
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Island

#### Displaying each neighborhood and its different venues

In [28]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Careful & Reliable Painting,43.752622,-79.331957,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...
2024,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2025,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2026,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2027,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


### Let's see the spreading of these venues on a map of Toronto

In [50]:
map_venues = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  
    
map_venues

In [30]:
toronto_venues.shape

(2029, 7)

#### Let's determine the number of venues for each neighborhood

In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
Weston,1,1,1,1,1,1
"Wexford, Maryvale",4,4,4,4,4,4
Willowdale,40,40,40,40,40,40


### Analyzing each neighborhood by revealing each category frequency of occurrence 

In [32]:
# one hot encoding
onehot_tor = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
onehot_tor['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [onehot_tor.columns[-1]] + list(onehot_tor.columns[:-1])
onehot_tor.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
onehot_tor.shape

(2029, 273)

#### Let's group rows by neighborhood

In [35]:
grouped_tor_neigh = onehot_tor.groupby('Neighborhood').mean().reset_index()
grouped_tor_neigh.head(5)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### Analyzing each Neighbourhood and his 5 top venues 

##### Only the first 10 neighborhoods have been displayed here to limit the scrolling. A better view is proposed below through a dataframe.

In [36]:
top_venues_count = 5

shape_count = 1
for quartier in grouped_tor_neigh['Neighborhood']:
    plist = grouped_tor_neigh[grouped_tor_neigh['Neighborhood'] == quartier].T.reset_index()
    plist.columns = ['venue','frequency']
    plist = plist.iloc[1:]
    plist['frequency'] = plist['frequency'].astype(float)
    plist = plist.round({'frequency': 2})
    if shape_count < 11:
        print("---- "+quartier+" ----"+" Ordre N°",shape_count)
        print(plist.sort_values('frequency', ascending=False).reset_index(drop=True).head(top_venues_count))
        print('\n')
    shape_count = shape_count + 1


---- Agincourt ---- Ordre N° 1
                       venue  frequency
0  Latin American Restaurant       0.25
1             Breakfast Spot       0.25
2                     Lounge       0.25
3               Skating Rink       0.25
4         Miscellaneous Shop       0.00


---- Alderwood, Long Branch ---- Ordre N° 2
                venue  frequency
0         Pizza Place        0.2
1                 Pub        0.1
2        Skating Rink        0.1
3      Sandwich Place        0.1
4  Athletics & Sports        0.1


---- Bathurst Manor, Wilson Heights, Downsview North ---- Ordre N° 3
           venue  frequency
0           Bank       0.09
1    Coffee Shop       0.09
2          Diner       0.05
3    Bridal Shop       0.05
4  Shopping Mall       0.05


---- Bayview Village ---- Ordre N° 4
                 venue  frequency
0  Japanese Restaurant       0.25
1                 Bank       0.25
2   Chinese Restaurant       0.25
3                 Café       0.25
4        Moving Target       0.00


-

In [37]:
def most_common_venues(row, top_venues_count):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues_count]

#### Let's create a new dataframe and send the data of the top 10 venues for each neighborhood in it.

In [38]:
top_venues_count = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues_count):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped_tor_neigh['Neighborhood']

for ind in np.arange(grouped_tor_neigh.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = most_common_venues(grouped_tor_neigh.iloc[ind, :], top_venues_count)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pub,Skating Rink,Gym,Dance Studio,Coffee Shop,Athletics & Sports,Sandwich Place,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Deli / Bodega
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Pizza Place,Sushi Restaurant,Restaurant,Café,Butcher,Japanese Restaurant,Thai Restaurant


## Methodology <a name="methodology"></a>


The purpose of this project is to help Noor find the best place for his restaurant. To achieve this goal, we tend to search areas where restaurants are not very present in the landscape. On the other end, those neighborhoods should be good neighborhoods in a sense of affluence and attraction.

On the step above, we've collected coordinates of different Toronto neighborhoods, and from this points, with the help of Foursquare database, we were able to gather the top 10 venues from each of these neighborhoods.

Now, in order to narrow the choice for Noor, we are going to use a clustering technique, called K-Means, to display neighborhoods that attracts a lot of people through their different commerces available, but also a low density of restaurants.

This is the methodology that will be used to provide the best recommendation to Noor.



In [71]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = grouped_tor_neigh.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 3, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 1,
       1, 1], dtype=int32)

In [40]:
grouped_tor_neigh

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.045455,0.000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Weston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
86,"Wexford, Maryvale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
87,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.025,0.0,0.0,0.0,0.0,0.0
88,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0


In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Food & Drink Shop,Park,Construction & Landscaping,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Pizza Place,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Arts & Crafts Store,Gift Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café


In [42]:
toronto_merged=toronto_merged.dropna()
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

### Let's visualize the clusters obtained

In [43]:
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [44]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,Park,Women's Store,Pool,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
64,York,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
85,Scarborough,Playground,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
91,Downtown Toronto,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
98,Etobicoke,Park,River,Pool,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


#### Cluster 2

In [45]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Food & Drink Shop,Park,Construction & Landscaping,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,Downtown Toronto,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Beer Store,Shoe Store
3,North York,Clothing Store,Accessories Store,Arts & Crafts Store,Gift Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,Downtown Toronto,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
7,North York,Gym,Coffee Shop,Asian Restaurant,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Italian Restaurant,Sporting Goods Shop,Bike Shop
...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,Coffee Shop,Café,Japanese Restaurant,Gym,Restaurant,Hotel,Steakhouse,Deli / Bodega,American Restaurant,Asian Restaurant
99,Downtown Toronto,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant,Yoga Studio,Hotel,Mediterranean Restaurant,Men's Store,Café
100,East Toronto,Light Rail Station,Yoga Studio,Garden,Skate Park,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Burrito Place,Butcher
101,Etobicoke,Construction & Landscaping,Deli / Bodega,Baseball Field,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


#### Cluster 3

In [46]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Pizza Place,Intersection,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center
50,North York,Pizza Place,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Hakka Restaurant
70,Etobicoke,Pizza Place,Intersection,Sandwich Place,Chinese Restaurant,Discount Store,Coffee Shop,Dog Run,Diner,Distribution Center,Doner Restaurant
77,Etobicoke,Pizza Place,Sandwich Place,Mobile Phone Shop,Bus Line,Dog Run,Diner,Discount Store,Distribution Center,Doner Restaurant,Farmers Market


#### Cluster 4

In [47]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,Fast Food Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
56,York,Fast Food Restaurant,Sandwich Place,Turkish Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Dessert Shop


## Final Analysis  <a name="analysis"></a>

Our analysis create the following five clusters of neighborhoods:

- __Cluster 1.__ This cluster is characterized by its high relative absence of restaurants, that could be a advantage. But the people around this cluster are interested in Yoga, Park, Playground, River, etc. They are not the right customers for a tropical and gastronomical restaurant as the one Noor want to open.
- __Cluster 2.__ This cluster is the larger one of all our clusters. It's characterized by many different categories of venues, within which restaurants are of them. A very cosmopolitan and heterogeneous area that addresses different communities. The area is not a residential one and although, the cluster may appear crowded by restaurants, its very large size makes it really possible for a new restaurant to settle. Moreover, the cluster provides many activities that can be opened at nights, once more fitting Noor's desire. This cluster shows many of the characteristics of what we are searching for. Neighborhoods like North York, Etibocoke or Downtown Toronto could be a good fit for Noor's restaurant.
- __Cluster 3.__ This cluster is characterized by the high density of restaurants of different kinds. There are a lot of choice for anyone searching for a table. Plus, the high number of fast-food restaurants inside this area could be a disadvantage for Noor's gastronomical restaurant because of the prices.
- __Cluster 4.__ This cluster is definitely not a good choice for Noor's restaurant as it contains already restaurants from different parts of the world, which is in fact the plus-value Noor intends to bring. Plus, Fast Food restaurants are registred as the 1st most common venue in this cluster, not a good point. Installing a tropical restaurant here could lead to a serious setback for Noor.


The result is the selection neighborhoods of Scarborough Village (Scarborough), Moore Park and Summerhill East (Central Toronto) as our recommendation for our stakeholdes due to their relative low presence of restaurants and a good combination of other main venues (also, not related to restaurants).


## Conclusion  <a name="conclusion"></a>

The aim of this project was to help Noor find a suitable place to settle his restaurant, a place that respect the conditions he wanted.
By retrieving the geospatial data of Toronto neighborhoods, collecting the most common venues of those neighborhoods and finally clustering the neighborhoods through their venues relativie composition, we were able to give a very accurate response to Noor's demands.

These are the neighborhoods have advised to Noor : __Downtown Toronto, Etobicoke__ and __North York__.

Now, it's Noor's call to to launch the next move based on our analysis, but also on some other factors like the level of the rental costs on these areas, for example, which was not part of our study.